# Confounding Robust Policy Evaluation
We evaluate the worst case regret of a policy $\pi$ relative to a baseline policy $\pi_0$ as presented in [Kallus et al.](https://arxiv.org/pdf/1805.08593.pdf)  


The model we are using:  
Medical Treatment model as defined in Appendix A in https://causalai.net/mdp-causal.pdf  
Here, there is a slight difference in physician's policy to ensure there is randomness in the physician's policy

In [1]:
import numpy as np
def physicains_policy(St, Mt, Et):
    p = 0.1
    # physician's policy is pi(St, Mt, Et) = St XOR Mt XOR Et XOR Z
    # where Z is Bern(p) random variable
    Z = np.random.binomial(1, p)
    return (St+Mt+Et+Z) % 2

In [2]:
# probability_yt_is_1 is a 4 dimensional array
# format: probability_yt_is_1[St][Mt][Et][Xt] is P(Yt = 1 | Xt, St, Mt, Et)
probability_yt_is_1 = np.zeros((2,2,2,2))
probability_yt_is_1[0][0][0][0] = 0.2
probability_yt_is_1[0][0][0][1] = 0.9
probability_yt_is_1[0][0][1][0] = 0.9
probability_yt_is_1[0][0][1][1] = 0.2
probability_yt_is_1[0][1][0][0] = 0.8
probability_yt_is_1[0][1][0][1] = 0.3
probability_yt_is_1[0][1][1][0] = 0.3
probability_yt_is_1[0][1][1][1] = 0.8

probability_yt_is_1[1][0][0][0] = 0.7
probability_yt_is_1[1][0][0][1] = 0.2
probability_yt_is_1[1][0][1][0] = 0.2
probability_yt_is_1[1][0][1][1] = 0.7
probability_yt_is_1[1][1][0][0] = 0.1
probability_yt_is_1[1][1][0][1] = 0.8
probability_yt_is_1[1][1][1][0] = 0.8
probability_yt_is_1[1][1][1][1] = 0.1

In [3]:
# transition_prob is a 2 dimensional array
# format: transition_prob[Xt][St] = P(St+1 = 0 | St, Xt)
transition_prob = np.zeros((2,2))
transition_prob[0][0] = 0.9
transition_prob[0][1] = 0.3
transition_prob[1][0] = 0.7
transition_prob[1][1] = 0.8

In [4]:
def get_Yt(St, Mt, Et, Xt):
    u = np.random.rand()
    if u < probability_yt_is_1[St][Mt][Et][Xt]:
        return 1
    return 0

In [5]:
def get_next_state(St, Xt):
    u = np.random.rand()
    if u < transition_prob[Xt][St]:
        return 0
    return 1

# Generate Data from model

In [6]:
import pandas as pd

data_df = pd.DataFrame()

In [7]:
def generate_single_trajectory(patient_id):
    global data_df
    St = np.random.randint(2)
    for t in range(100):
        Mt, Et = np.random.randint(2), np.random.randint(2)
        Xt = physicains_policy(St, Mt, Et)
        Yt = get_Yt(St, Mt, Et, Xt)
        data_df = data_df.append({'pt_id': patient_id,'t': t, 'St': St, 'Mt': Mt, 'Et': Et, 'Xt': Xt, 'Yt': Yt}, ignore_index=True)
        St = get_next_state(St, Xt)

In [ ]:
for patient_id in range(5000):
    if patient_id % 500 == 0:
        print("Iteration number: ", patient_id)
    generate_single_trajectory(patient_id)

We simply reuse the data already generated before

In [10]:
data_df = pd.read_csv("/Users/faaiz/MDPUC/data-model2.csv")

### Divide the data into training and testing sets

In [11]:
patients = data_df['pt_id'].unique()
training = patients[np.random.randint(5, size = (len(patients))) != 4]
testing = patients[np.random.randint(5, size = (len(patients))) == 4]

train_data = data_df.loc[data_df['pt_id'].isin(training)].reset_index()
test_data = data_df.loc[data_df['pt_id'].isin(testing)].reset_index()

### Compute the physician's policy. 
We will use this as the baseline $\pi_0$ when calculating the worst case regret for our policy.

In [14]:
from util_functions import *

sums = compute_state_action_visits(train_data, n_states=2, n_actions=2)

In [15]:
physpol = (sums.T/((sums.sum(axis=1)==0) + (sums.sum(axis=1)))).T

Our policy $\pi$ is obtained by 'softening' the deterministic policy  
$\pi(X_t = 0 | S_t = 0) = 1$  
$\pi(X_t = 1 | S_t = 1) = 1$

In [17]:
pi = np.zeros((2,2))
pi[0][0] = 0.95
pi[0][1] = 0.05
pi[1][0] = 0.05
pi[1][1] = 0.95
pi

array([[0.95, 0.05],
       [0.05, 0.95]])

### Estimating nominal propensities $P[X_t = x | S_t = s]$ from data
We can just estimate this from the physician's policy

In [19]:
propensity = physpol

For the purposes of this analysis, we assume $\Gamma = 1.2$. We will calibrate the value of $\Gamma$ when doing a more careful analysis.  
Here $\Gamma$ is defined as a bound $\Gamma\geq1$, such that:  
$\Gamma^{-1}\leq\frac{(1-\tilde{e}_T(X))e_T(X,Y)}{\tilde{e}_T(X)(1-e_T(X,Y))}\leq\Gamma$  
here $\tilde{e}_T(X) = P(T=t | X=x)$  
and $e_T(X,Y)=P(T=t | X=x, Y(t) = y)$

We use **Theorem 11** in [Kallus et al.](https://arxiv.org/pdf/1805.08593.pdf) to calculate the worst case regret. In order to do this, we must first compute $r_i$, $a_i^\Gamma$ and $b_i^\Gamma$ values as defined in the paper.

In [22]:
gamma = 1.2
cr_data = train_data.copy()
for index, row in cr_data.iterrows():
    cr_data.at[index, 'r'] = (physpol[int(row['St'])][int(row['Xt'])]-pi[int(row['St'])][int(row['Xt'])])*row['Yt']
    cr_data.at[index, 'a'] = 1 + 1/gamma*(1/propensity[int(row['St'])][int(row['Xt'])] - 1)
    cr_data.at[index, 'b'] = 1 + gamma*(1/propensity[int(row['St'])][int(row['Xt'])] - 1)
    

In [23]:
cr_data.head()

,index,Unnamed: 0,Et,Mt,St,Xt,Yt,pt_id,t,r,a,b
0,0,0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.44825,1.839187,2.208430
1,1,1,1.0,1.0,0.0,0.0,0.0,0.0,1.0,-0.00000,1.827520,2.191629
2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-0.00000,1.827520,2.191629
3,3,3,1.0,0.0,0.0,1.0,0.0,0.0,3.0,0.00000,1.839187,2.208430
4,4,4,0.0,1.0,0.0,1.0,0.0,0.0,4.0,0.00000,1.839187,2.208430


In [33]:
cr_data_sorted = cr_data.sort_values(by='r').reset_index()

In [34]:
cr_data_sorted.head()

,level_0,index,Unnamed: 0,Et,Mt,St,Xt,Yt,pt_id,t,r,a,b
0,249328,308928,308928,0.0,0.0,1.0,1.0,1.0,3089.0,28.0,-0.451754,1.839202,2.208451
1,290255,360855,360855,0.0,0.0,1.0,1.0,1.0,3608.0,55.0,-0.451754,1.839202,2.208451
2,215491,267191,267191,1.0,1.0,1.0,1.0,1.0,2671.0,91.0,-0.451754,1.839202,2.208451
3,390099,487999,487999,0.0,0.0,1.0,1.0,1.0,4879.0,99.0,-0.451754,1.839202,2.208451
4,157382,194582,194582,0.0,0.0,1.0,1.0,1.0,1945.0,82.0,-0.451754,1.839202,2.208451


In [49]:
def compute_lambda(k, cr_data_sorted):
    num = ((cr_data_sorted.index<k)*(cr_data_sorted['a']*cr_data_sorted['r'])).sum() +\
    ((cr_data_sorted.index>=k)*(cr_data_sorted['b']*cr_data_sorted['r'])).sum()
    den = ((cr_data_sorted.index<k)*cr_data_sorted['a']).sum() +\
    ((cr_data_sorted.index>=k)*cr_data_sorted['b']).sum()
    return num/den    

As proved in the paper, the worst case regret estimate $\hat{\bar{Q}}(r;W_n^\Gamma) = \lambda(k^*)$  
where $k^* = \inf\{k=1,\dots,n+1:\lambda(k)<\lambda(k-1)\}$

In [51]:
lbd = compute_lambda(0, cr_data_sorted)
next_lbd = compute_lambda(1, cr_data_sorted)
i = 2
while i < len(cr_data_sorted) and  lbd <= next_lbd:
    lbd = next_lbd
    next_lbd = compute_lambda(i, cr_data_sorted)
    i += 1
if next_lbd < lbd:
    print('Worst case regret is', next_lbd)
else:
    print('k* is infinity')

Worst case regret is 0.012713775195430332


# Bring this all together in a function 

In [55]:
def compute_worst_case_regret(data, pi, pi0, gamma):
    n_states = len(pi)
    n_actions = len(pi[0])
    sums = compute_state_action_visits(data, n_states, n_actions)
    physpol = (sums.T/((sums.sum(axis=1)==0) + (sums.sum(axis=1)))).T
    propensity = physpol
    cr_data = data.copy()
    for index, row in cr_data.iterrows():
        cr_data.at[index, 'r'] = (physpol[int(row['St'])][int(row['Xt'])]-pi[int(row['St'])][int(row['Xt'])])*row['Yt']
        cr_data.at[index, 'a'] = 1 + 1/gamma*(1/propensity[int(row['St'])][int(row['Xt'])] - 1)
        cr_data.at[index, 'b'] = 1 + gamma*(1/propensity[int(row['St'])][int(row['Xt'])] - 1)
    cr_data_sorted = cr_data.sort_values(by='r').reset_index()
    lbd = compute_lambda(0, cr_data_sorted)
    next_lbd = compute_lambda(1, cr_data_sorted)
    i = 2
    while i < len(cr_data_sorted) and  lbd <= next_lbd:
        lbd = next_lbd
        next_lbd = compute_lambda(i, cr_data_sorted)
        i += 1
    if next_lbd < lbd:
        print('Worst case regret is', next_lbd)
        return next_lbd
    else:
        print('k* is infinity')
        return None

Let's try the same with policy $\pi$ obtained by 'softening' the deterministic policy  
$\pi(X_t = 1 | S_t = 0) = 1$  
$\pi(X_t = 0 | S_t = 1) = 1$

In [60]:
pi = np.zeros((2,2))
pi[0][0] = 0.05
pi[0][1] = 0.95
pi[1][0] = 0.95
pi[1][1] = 0.05
pi

array([[0.05, 0.95],
       [0.95, 0.05]])

In [61]:
compute_worst_case_regret(train_data, pi, physpol, gamma=1.2)

Worst case regret is 0.012713775195430332


0.012713775195430332

### Change the degree of unmeasured confounding
Now let's assume that we only have access to $S_t$ and $E_t$. $M_t$ is an unobserved confounder. In this case, the state space has 4 states. The action space remains unchanged.

In [65]:
train_data_model2 = train_data.copy()
train_data_model2.head()

,index,Unnamed: 0,Et,Mt,St,Xt,Yt,pt_id,t
0,0,0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
1,1,1,1.0,1.0,0.0,0.0,0.0,0.0,1.0
2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,3,3,1.0,0.0,0.0,1.0,0.0,0.0,3.0
4,4,4,0.0,1.0,0.0,1.0,0.0,0.0,4.0


In [66]:
train_data_model2['St'] = train_data_model2['St'] + 2*train_data_model2['Et']
train_data_model2.drop(columns = ['Et', 'Mt'], inplace=True)
train_data_model2.head()

,index,Unnamed: 0,St,Xt,Yt,pt_id,t
0,0,0,2.0,1.0,1.0,0.0,0.0
1,1,1,2.0,0.0,0.0,0.0,1.0
2,2,2,0.0,0.0,0.0,0.0,2.0
3,3,3,2.0,1.0,0.0,0.0,3.0
4,4,4,0.0,1.0,0.0,0.0,4.0


### Optimal policy $\pi^*$
We know from the complete model dynamics that the optimal policy $\pi^*$ in this case is:  
$(S_t, E_t)$ -> Optimal Action  
(0,0) -> 1  
(0,1) -> 0  
(1,0) -> 1  
(1,1) -> 0

In [72]:
def soften_deterministic_policy(policy):
    p = 0.05
    n_states = len(policy)
    n_actions = len(policy[0])
    for i in range(len(policy)):
        for j in range(len(policy[i])):
            if policy[i][j] == 0:
                policy[i][j] = p/(n_actions-1)
            elif policy[i][j] == 1:
                policy[i][j] = 1-p
    return policy

A policy which always picks action 0 must have higher regret than the optimal policy $\pi^*$ relative to the physician's policy.

In [73]:
pi = np.zeros((4,2))
pi[0][0] = 1
pi[1][0] = 1
pi[2][0] = 1
pi[3][0] = 1
soften_deterministic_policy(pi)

array([[0.95, 0.05],
       [0.95, 0.05],
       [0.95, 0.05],
       [0.95, 0.05]])

In [74]:
sums = compute_state_action_visits(train_data_model2, n_states=4, n_actions=2)
physpol = (sums.T/((sums.sum(axis=1)==0) + (sums.sum(axis=1)))).T
physpol

array([[0.5002815 , 0.4997185 ],
       [0.50353386, 0.49646614],
       [0.50320865, 0.49679135],
       [0.49998152, 0.50001848]])

In [ ]:
compute_worst_case_regret(train_data_model2, soften_deterministic_policy(pi), physpol, gamma=1.2)

In [ ]:
pi_star = np.zeros((4,2))
pi_star[0][1] = 1
pi_star[1][1] = 1
pi_star[2][0] = 1
pi_star[3][0] = 1

In [ ]:
compute_worst_case_regret(train_data_model2, soften_deterministic_policy(pi_str), physpol, gamma=1.2)